In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
!pip3 install transformers
!pip3 install simpletransformers
import pandas as pd
import logging
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 42.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.7/250.7 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 5.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 106.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/Electra_Trained.zip" -d "/content/electra"
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/MedElectra_Trained.zip" -d "/content/medelectra"
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/Roberta_Trained.zip" -d "/content/roberta"
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/biobert_2_epch.zip" -d "/content/biobert"
!unzip "/content/drive/MyDrive/472 Project Files/Trained Models/saved_model_distil_ro_berta.zip" -d "/content/distilled_roberta"

Archive:  /content/drive/MyDrive/472 Project Files/Trained Models/Electra_Trained.zip
   creating: /content/electra/content/outputs/checkpoint-58284-epoch-4/
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/training_args.bin  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/config.json  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/pytorch_model.bin  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/tokenizer_config.json  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/scheduler.pt  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/vocab.txt  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/optimizer.pt  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/model_args.json  
  inflating: /content/electra/content/outputs/checkpoint-58284-epoch-4/special_tokens_map.json  
  inflating: /content/electra/content/o

In [ ]:
import os

def get_model_folder(model_name, base_folder="/content/"):
  location = f"{base_folder}{model_name}/content/outputs"
  checkpoint_folder = os.listdir(location)[0]
  return f"{location}/{checkpoint_folder}"

In [ ]:
#Load Electra
model_args = ClassificationArgs(num_train_epochs=4)

modelElectra = ClassificationModel("electra", get_model_folder("electra"), num_labels = 40, args=model_args, use_cuda = True)

In [ ]:
#Load MedElectra
model_args = ClassificationArgs(num_train_epochs=4)

modelMedElectra = ClassificationModel("electra", get_model_folder("medelectra"), num_labels = 40, args=model_args, use_cuda = True)

In [ ]:
#Load Roberta
model_args = ClassificationArgs(num_train_epochs=4)

modelRoBERTa = ClassificationModel("roberta", get_model_folder("roberta"), num_labels = 40, args=model_args, use_cuda = True)

In [ ]:
#Load Distilled Roberta
model_args = ClassificationArgs(num_train_epochs=4)

modelDistilledRoBERTa = ClassificationModel("roberta", get_model_folder("distilled_roberta"), num_labels = 40, args=model_args, use_cuda = True)

In [ ]:
#Load BioBert
model_args = ClassificationArgs(num_train_epochs=4)

modelBioBERT = ClassificationModel("bert", get_model_folder("biobert"), num_labels = 40, args=model_args, use_cuda = True)

In [ ]:
eval_df = pd.read_csv('/content/drive/MyDrive/472 Project Files/Stratified Input Data/eval_df-3.csv')
eval_df.dropna(how='all')
eval_df["text"] = eval_df["text"].astype(str)
val3= eval_df['text'].tolist()


In [ ]:
import numpy as np

def analyse(model):

  first_acc= 0
  second_acc= 0 
  third_acc= 0
  final_acc= 0

  label_counts = np.zeros(40, dtype = int)

  label_accuracies = []
  for i in range(40):
    label_accuracies.append([0,0,0])

  predictions, raw_outputs = model.predict(val3)
  eval_df.columns= ['text','labels']

  for x in range(len(raw_outputs)):
    indexes = sorted(range(len(raw_outputs[x])), key=lambda i: raw_outputs[x][i], reverse=True)
    if indexes[0] == eval_df["labels"][x]:
      first_acc+=1
      label_accuracies[indexes[0]][0] += 1
    elif indexes[1] == eval_df["labels"][x]:
      second_acc+=1
      label_accuracies[indexes[1]][1] += 1
    elif indexes[2] == eval_df["labels"][x]:
      third_acc+=1
      label_accuracies[indexes[2]][2] += 1

    
    

  l_all= len(eval_df)
  first_acc/= l_all
  second_acc/= l_all
  third_acc/= l_all
  final_acc= first_acc + second_acc + third_acc

  print(f'First Accuracy: {first_acc}')
  print(f'Second Accuracy: {second_acc + first_acc}')
  print(f'Third Accuracy: {final_acc}')
  print(f'Final Accuracy: {final_acc}')
  return(first_acc,second_acc,third_acc,final_acc,label_accuracies,label_counts)

In [ ]:
ElectraResults = analyse(modelElectra)

  0%|          | 0/38856 [00:00<?, ?it/s]

  0%|          | 0/4857 [00:00<?, ?it/s]

First Accuracy: 0.33284434836318716
Second Accuracy: 0.17639489396746963
Third Accuracy: 0.08302450072060943
Final Accuracy: 0.5922637430512662


In [ ]:
MedElectraResults = analyse(modelMedElectra)

  0%|          | 0/38856 [00:00<?, ?it/s]

  0%|          | 0/4857 [00:00<?, ?it/s]

First Accuracy: 0.3315832818612312
Second Accuracy: 0.2225396335186329
Third Accuracy: 0.09532633312744493
Final Accuracy: 0.6494492485073091


In [ ]:
RobertaResults = analyse(modelRoBERTa)

  0%|          | 0/38856 [00:00<?, ?it/s]

  0%|          | 0/4857 [00:00<?, ?it/s]

First Accuracy: 0.34169754992793905
Second Accuracy: 0.604282478896438
Third Accuracy: 0.7150504426600781
Final Accuracy: 0.7150504426600781


In [ ]:
DistilledRobertaResults = analyse(modelDistilledRoBERTa)

  0%|          | 0/38856 [00:00<?, ?it/s]

  0%|          | 0/4857 [00:00<?, ?it/s]

First Accuracy: 0.33732242124768375
Second Accuracy: 0.26451513279802347
Third Accuracy: 0.11233786287831996
Final Accuracy: 0.7141754169240272


In [ ]:
BioBERTResults = analyse(modelBioBERT)

  0%|          | 0/38856 [00:00<?, ?it/s]

  0%|          | 0/4857 [00:00<?, ?it/s]

First Accuracy: 0.3265904879555281
Second Accuracy: 0.29146077825818406
Third Accuracy: 0.1200072060942969
Final Accuracy: 0.738058472308009


In [ ]:
inf_df = pd.DataFrame()

def analysisResults(result):
  no_counts = []
  prediction_percents = []
  x = 0;
  for i in result[4]:
    if i == [0,0,0]:
      no_counts.append(x)
      prediction_percents.append(0)
    else:
      n = eval_df['labels'].value_counts()[x]
      prediction_percents.append(i[0]/n)
    x+=1

  print(len(no_counts))
  print(no_counts)
  print(prediction_percents)
  print(sum(prediction_percents)/len(prediction_percents))
  return(no_counts,prediction_percents,sum(prediction_percents)/len(prediction_percents))

In [ ]:
#Electra
#out = analysisResults(ElectraResults)
#inf_df["Electra"] = out[1]
#print(ElectraResults[5][3]/sum(ElectraResults[5]))
#print(ElectraResults[5])

print(ElectraResults[0])
print(ElectraResults[0] + ElectraResults[1])
print(ElectraResults[3])

0.33284434836318716
0.5092392423306568
0.5922637430512662


In [ ]:
#MedElectra
# out = analysisResults(MedElectraResults)
# inf_df["MedElectra"] = out[1]
# print(MedElectraResults[5][3]/sum(MedElectraResults[5]))
# print(MedElectraResults[5])

print(MedElectraResults[0])
print(MedElectraResults[0] + MedElectraResults[1])
print(MedElectraResults[3])

0.3315832818612312
0.5541229153798641
0.6494492485073091


In [ ]:
#Roberta
out = analysisResults(RobertaResults)
inf_df["Roberta"] = out[1]

# print(RobertaResults[0])
# print(RobertaResults[0] + RobertaResults[1])
# print(RobertaResults[3])

3
[0, 2, 17]
[0, 0.5306122448979592, 0, 0.2847372210223182, 0.04950495049504951, 0.6410397217645982, 0.015151515151515152, 0.18932038834951456, 0.1024390243902439, 0.2545454545454545, 0.30198019801980197, 0.024390243902439025, 0.07777777777777778, 0.16792738275340394, 0.21661931818181818, 0.066746126340882, 0.0672782874617737, 0, 0.2926829268292683, 0.06451612903225806, 0.12574850299401197, 0.10191082802547771, 0.34796404019037547, 0.10951760104302477, 0.2964143426294821, 0.09904153354632587, 0.20518358531317496, 0.27926749509483323, 0.271889400921659, 0.0992, 0.11564625850340136, 0.06567164179104477, 0.29191616766467066, 0.22148148148148147, 0.0, 0.23711340206185566, 0.12014134275618374, 0.3023255813953488, 0.5850586181092542, 0.16863207547169812]
0.18478482024773446


In [ ]:
#Distilled Roberta
# out = analysisResults(DistilledRobertaResults)
# inf_df["Distilled Roberta"] = out[1]
print(DistilledRobertaResults[0])
print(DistilledRobertaResults[0] + DistilledRobertaResults[1])
print(DistilledRobertaResults[3])

0.33732242124768375
0.6018375540457073
0.7141754169240272


In [ ]:
#biobert
out = analysisResults(BioBERTResults)
# inf_df["BioBert"] = out[1]
# inf_df['freq']= eval_df["label"].value_counts()

# print(BioBERTResults[0])
# print(BioBERTResults[0] + BioBERTResults[1])
# print(BioBERTResults[3])

2
[2, 17]
[0.0, 0.6122448979591837, 0, 0.30093592512598993, 0.06930693069306931, 0.5848434925864909, 0.06565656565656566, 0.18932038834951456, 0.1073170731707317, 0.12727272727272726, 0.29207920792079206, 0.03780487804878049, 0.06111111111111111, 0.1800302571860817, 0.14204545454545456, 0.07667858561779897, 0.11314984709480122, 0, 0.23577235772357724, 0.16129032258064516, 0.11377245508982035, 0.1337579617834395, 0.33897408778424115, 0.14602346805736635, 0.37290836653386455, 0.07667731629392971, 0.4190064794816415, 0.3623283191628515, 0.4930875576036866, 0.08, 0.1292517006802721, 0.10149253731343283, 0.3218562874251497, 0.16, 0.05747126436781609, 0.3402061855670103, 0.127208480565371, 0.23255813953488372, 0.4985033674232976, 0.07900943396226415]
0.19852383578184138


In [ ]:
inf_df.head(40)

In [ ]:
eval_df["label"].value_counts()

In [ ]:
in_df = pd.read_csv("/content/drive/MyDrive/472 Project Files/Stratified Input Data/eval_df-3.csv")

in_df["label"].value_counts()

38    8018
5     5463
27    3058
3     2778
15    2517
22    1891
14    1408
33    1350
24    1255
36    1132
39     848
11     820
10     808
23     767
32     668
13     661
16     654
21     628
29     625
26     463
31     335
25     313
18     246
28     217
7      206
8      205
4      202
6      198
12     180
20     167
30     147
1      147
35      97
34      87
17      64
0       56
9       55
2       48
37      43
19      31
Name: label, dtype: int64

In [ ]:
in_df = pd.read_csv("/content/mtsamples (2).csv")

#in_df.head()

in_df["medical_specialty"].value_counts()


 Surgery                          1103
 Consult - History and Phy.        516
 Cardiovascular / Pulmonary        372
 Orthopedic                        355
 Radiology                         273
 General Medicine                  259
 Gastroenterology                  230
 Neurology                         223
 SOAP / Chart / Progress Notes     166
 Obstetrics / Gynecology           160
 Urology                           158
 Discharge Summary                 108
 ENT - Otolaryngology               98
 Neurosurgery                       94
 Hematology - Oncology              90
 Ophthalmology                      83
 Nephrology                         81
 Emergency Room Reports             75
 Pediatrics - Neonatal              70
 Pain Management                    62
 Psychiatry / Psychology            53
 Office Notes                       51
 Podiatry                           47
 Dermatology                        29
 Cosmetic / Plastic Surgery         27
 Dentistry               